In [ ]:
import math
import numpy as np
import random
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
goal_states = [1, 3]
size_of_maze = 4
episodes = 10000

discount = 0.5
alpha = 0.01
e_soft = 0.01

n = 1024

inputs = n
ouputs = 1

reward_bad = 0
reward_good = 1

threshold = -0.35

In [ ]:
def hrr(length, normalized=False):
    if normalized:
        x = np.random.uniform(-np.pi,np.pi,int((length-1)/2))
        if length % 2:
            x = np.real(np.fft.ifft(np.concatenate([np.ones(1), np.exp(1j*x), np.exp(-1j*x[::-1])])))
        else:
            x = np.real(np.fft.ifft(np.concatenate([np.ones(1), np.exp(1j*x), np.ones(1), np.exp(-1j*x[::-1])])))
    else:
        x = np.random.normal(0.0, 1.0/np.sqrt(length), length)
    return x

In [ ]:
state_hrrs = np.zeros([size_of_maze, n])
for x in range(size_of_maze):
    state_hrrs[x] = hrr(n, True)
    
color_hrrs_internal = np.zeros([len(goal_states), n])
for x in range(len(goal_states)):
    color_hrrs_internal[x] = hrr(n, True)
possible_wm = color_hrrs_internal

reward_tkn = hrr(n, True)

In [ ]:
def convolve(x, y):
    return np.real(np.fft.ifft(np.fft.fft(x)*np.fft.fft(y)))
def preconvolve():
    preconvolved_matrix = np.zeros([possible_wm.size, size_of_maze, n])
    for x in range(len(possible_wm)):
        for y in range(size_of_maze):
            preconvolved_matrix[x][y] = convolve(possible_wm[x], state_hrrs[y])
    return preconvolved_matrix

In [ ]:
def policy_mov(state_left, state_right, wm, rand_on):
    value_left_state_wm = nn.feedforward(preconvolved_matrix[wm][state_left])
    value_right_state_wm = nn.feedforward(preconvolved_matrix[wm][state_right])
    if((np.random.random() < e_soft) and (rand_on == 1)):
        return np.random.choice([-1, 1])
    max_value = max(value_left_state_wm, value_right_state_wm)
    if(max_value == value_left_state_wm):
        return -1
    elif(max_value == value_right_state_wm):
        return 1
    
def policy_switch(state, wm, color, error):
    return (wm + 1)%2

In [ ]:
class NeuralNetwork:
    def __init__(self, inputs, outputs, discount, alpha):
        self.discount = discount
        self.alpha = alpha
        self.input = inputs
        self.output = outputs
        self.weights = hrr(n, True)
        self.bias = 0
    def feedforward(self, X):
        self.output = np.dot(self.weights, X) + self.bias
        return self.output
    def backprop(self, state_prime_hrr, state_hrr, wm, y):
        if(y == reward_good):
            self.error = reward_good - self.feedforward(state_hrr)
            self.weights = np.add(self.weights, (self.alpha * self.error * state_hrr))
        else:
            self.error = (reward_bad + self.discount * self.feedforward(state_hrr)) - self.feedforward(state_prime_hrr)
            if self.error < threshold:
                wm = policy_switch(state, wm, color, self.error)
                return wm
            self.weights = np.add(self.weights, (self.alpha * self.error * state_prime_hrr))
        return wm

In [ ]:
preconvolved_matrix = preconvolve()
nn = NeuralNetwork(n, 1, discount, alpha)

In [ ]:
state = random.randint(0, size_of_maze - 1)
state_prime = state
steps = 0
count = 0
wm = 0
for x in range(episodes):
        steps = 0
        count += 1
        if count <= 10:
            if count == 1:
                goal_state = goal_states[0]
            pass
        elif count > 10 and count < 20:
            if count == 11:
                goal_state = goal_states[1]
            pass
        elif count >= 20:
            count = 0
        while(steps<30):
            steps += 1
            if(state == 0):
                decision = policy_mov(size_of_maze - 1, 1, wm, 1)
                if(decision == -1):
                    state_prime = state
                    state = size_of_maze - 1
                else:
                    state_prime = state
                    state = state + decision
            elif(state == size_of_maze - 1):
                decision = policy_mov(state - 1, 0, wm, 1)
                if(decision == -1):
                    state_prime = state
                    state = state + decision
                else:
                    state_prime = state
                    state = 0
            else:
                decision = policy_mov(state - 1, state + 1, wm, 1)
                state_prime = state
                state = state + decision
            old_wm = wm
            wm = nn.backprop(preconvolved_matrix[old_wm][state_prime], preconvolved_matrix[wm][state], wm, reward_bad)
            if state == goal_state:
                nn.backprop(preconvolved_matrix[old_wm][state_prime], convolve(reward_tkn, preconvolved_matrix[wm][state]), wm, reward_good)
                break
        state = random.randint(0, size_of_maze - 1)
        state_prime = state
        print("Episode", x+1, "done")

In [ ]:
position = np.arange(size_of_maze)
value = np.zeros(size_of_maze)
for z in range (size_of_maze):
    if z == 1:
        value[z] = nn.feedforward(convolve(reward_tkn, preconvolved_matrix[0][z]))
    else:
        value[z] = nn.feedforward(preconvolved_matrix[0][z])
plt.plot(position, value)
print(np.amax(value))

In [ ]:
position = np.arange(size_of_maze)
value = np.zeros(size_of_maze)
for z in range (size_of_maze):
    if z == 3:
        value[z] = nn.feedforward(convolve(reward_tkn, preconvolved_matrix[0][z]))
    else:
        value[z] = nn.feedforward(preconvolved_matrix[0][z])
plt.plot(position, value) 
print(np.amax(value))